In [ ]:
!apt-get install poppler

In [ ]:
#need to install both the tesseract package for linux, along with the Python binding
! apt install tesseract-ocr
! apt install libtesseract-dev
!pip install pytesseract
!pip install pillow
!pip install opencv-python
!pip install pandas
!pip install pdf2image
!apt-get install poppler-utils

In [ ]:
!pip install PyPDF2

In [ ]:
import cv2
import pytesseract
import pandas as pd
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow
from PyPDF2 import PdfReader, PdfWriter
from pdf2image import convert_from_path

In [ ]:
def crop_pdf(input_pdf, output_pdf, x1, y1, x2, y2):
    with open(input_pdf, 'rb') as file:
        reader = PdfReader(file)
        writer = PdfWriter()

        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            page.mediabox.lower_left = (x1, y1)
            page.mediabox.upper_right = (x2, y2)
            writer.add_page(page)

        with open(output_pdf, 'wb') as output_file:
            writer.write(output_file)

    print("PDF cropped successfully.")

# Specify the input PDF file path
input_pdf = '/content/crop_image.pdf'

# Specify the output PDF file path
output_pdf = '/content/cropped_image.pdf'

# Specify the coordinates for cropping (in points)
# Size for linerlytica page 6 SHFI table
x1 = 40
y1 = 510
x2 = 570
y2 = 683

# Crop the PDF
crop_pdf(input_pdf, output_pdf, x1, y1, x2, y2)

PDF cropped successfully.


In [ ]:
def convert_pdf_to_png(input_path, output_path):
    pages = convert_from_path(input_path)

    for i, page in enumerate(pages):
        page.save(f"{output_path}.png", "PNG")

    print("PDF converted to PNG images successfully.")

# Specify the input PDF file path
input_path = '/content/cropped_image.pdf'

# Specify the output directory path where PNG images will be saved
output_path = '/content/cropped_image'

# Convert PDF to PNG images
convert_pdf_to_png(input_path, output_path)

PDF converted to PNG images successfully.


In [ ]:
# Load the screenshot image
image_path = f"{output_path}.png"
image = Image.open(image_path)

In [ ]:
# Convert the image to a NumPy array
image_np = np.array(image)

In [ ]:
# Preprocess the image to enhance the text extraction using OpenCV
image_gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
image_thresh = cv2.threshold(image_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [ ]:
## Use OCR to extract the text from the preprocessed image
extracted_text = pytesseract.image_to_string(image)

In [ ]:
## Process the extracted text to convert it into a structured table format
table_data = [row.strip().split() for row in extracted_text.split('\n') if row.strip()]
df = pd.DataFrame(table_data)

In [ ]:
# Calculate the number of columns based on the desired arrangement
num_rows = 13
num_columns = len(table_data) // num_rows

In [ ]:
# Create a dictionary to store the table data for each column
table_columns = {}
for i in range(num_columns):
    column_data = table_data[i*num_rows : (i+1)*num_rows]
    table_columns[f'Column {i+1}'] = column_data

In [ ]:
# Create a DataFrame from the table columns
df = pd.DataFrame(table_columns)

# Access the extracted table data stored in the DataFrame df for further analysis or export
df